# Add memory
- Doc: https://langchain-ai.github.io/langgraph/tutorials/get-started/3-add-memory/

In [ ]:
from dotenv import load_dotenv

# API-KEY 읽어오기
load_dotenv()

In [ ]:
from langchain.chat_models import init_chat_model

# 모델 초기화
llm = init_chat_model("google_genai:gemini-2.5-flash")

## 1. Create a MemorySaver checkpointer

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# MemorySaver: 워크플로우의 상태를 메모리에 저장
memory = MemorySaver()

## 2. Compile the graph

In [ ]:
from typing import Annotated

from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


# 에이전트의 상태 정의 클래스
class State(TypedDict):
    messages: Annotated[list, add_messages]

# 상태 기반 워크플로우 생성
graph_builder = StateGraph(State)

# 웹 검색을 도구
tool = TavilySearch(max_results=2)
# 도구 리스트
tools = [tool]

# LLM이 도구 호출 여부 판단
llm_with_tools = llm.bind_tools(tools)

# chatbot 노드 함수
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# 워크플로우에 chatbot 노드 추가
graph_builder.add_node("chatbot", chatbot)

# LangGraph의 prebuilt ToolNode를 사용
tool_node = ToolNode(tools=[tool])
# tool 노드 워크플로우에 추가
graph_builder.add_node("tools", tool_node)

# 조건부 라우팅: tools로 이동하거나 END로 이동 (종료)
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,  # LangGraph의 prebuilt 함수
                      # 마지막 메시지에 `tool_calls`가 있는지 확인
)

# tools 노드 실행 후 chatbot 노드로 다시 이동 (도구 결과 처리)
graph_builder.add_edge("tools", "chatbot")
# 워크플로우 시작점에서 chatbot 노드로 이동
graph_builder.add_edge(START, "chatbot")

In [ ]:
# 워크플로우 컴파일, 실행 가능한 워크플로우 생성.
# memory checkpointer 사용
graph = graph_builder.compile(checkpointer=memory)

In [ ]:
from IPython.display import Image, display

try:
    # 워크플로우 그래프를 이미지로 시각화하여 출력
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

## 3. Interact with your chatbot

In [ ]:
# 대화 세션을 구분하기 위한 설정
# thread_id는 대화의 고유 식별자로, MemorySaver가 대화 기록을 저장/로드하는 데 사용
config = {"configurable": {"thread_id": "1"}}

def stream_graph_updates(user_input: str):
    # 워크플로우를 스트리밍 모드로 실행 (사용자 입력 처리)
    events = graph.stream(
        {"messages": [{"role": "user", "content": user_input}]},
        config=config
    )
    for event in events:
        for value in event.values():
            # 이벤트의 마지막 메시지 출력
            value["messages"][-1].pretty_print()


while True:
    try:
        user_input = input("User: ")  # 사용자 입력 받기
        if user_input.lower() in ["quit", "exit", "q"]:  # 종료 조건
            print("Goodbye!")
            break

        # 워크플로우 실행
        stream_graph_updates(user_input)
    except:
        # 오류 발생시 기본 질문으로 대체
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

## 4. Ask a follow up question

In [ ]:
# 기존 thread_id 사용 (이전 대화 세션)
config = {"configurable": {"thread_id": "1"}}

def stream_graph_updates(user_input: str):
    # 워크플로우를 스트리밍 모드로 실행 (사용자 입력 처리)
    events = graph.stream(
        {"messages": [{"role": "user", "content": user_input}]},
        config=config
    )
    for event in events:
        for value in event.values():
            # 이벤트의 마지막 메시지 출력
            value["messages"][-1].pretty_print()


while True:
    try:
        user_input = input("User: ")  # 사용자 입력 받기
        if user_input.lower() in ["quit", "exit", "q"]:  # 종료 조건
            print("Goodbye!")
            break

        # 워크플로우 실행
        stream_graph_updates(user_input)
    except:
        # 오류 발생시 기본 질문으로 대체
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

In [ ]:
# 새로운 thread_id 사용 (새로운 대화 세션)
config = {"configurable": {"thread_id": "2"}}

def stream_graph_updates(user_input: str):
    # 워크플로우를 스트리밍 모드로 실행 (사용자 입력 처리)
    events = graph.stream(
        {"messages": [{"role": "user", "content": user_input}]},
        config=config
    )
    for event in events:
        for value in event.values():
            # 이벤트의 마지막 메시지 출력
            value["messages"][-1].pretty_print()


while True:
    try:
        user_input = input("User: ")  # 사용자 입력 받기
        if user_input.lower() in ["quit", "exit", "q"]:  # 종료 조건
            print("Goodbye!")
            break

        # 워크플로우 실행
        stream_graph_updates(user_input)
    except:
        # 오류 발생시 기본 질문으로 대체
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

## 5. Inspect the state

In [ ]:
# thread_id: 1
config = {"configurable": {"thread_id": "1"}}

# config에 해당하는 워크플로우의 상태 스냅샷 조회
snapshot = graph.get_state(config)

print("-" * 20, "messages", "-" * 20)
for v in snapshot.values['messages']:
    print(v)
print("-" * 20, "config", "-" * 20)
print(snapshot.config)
print("-" * 20, "next", "-" * 20)
print(snapshot.next)

In [ ]:
# thread_id: 2
config = {"configurable": {"thread_id": "2"}}

# config에 해당하는 워크플로우의 상태 스냅샷 조회
snapshot = graph.get_state(config)

print("-" * 20, "messages", "-" * 20)
for v in snapshot.values['messages']:
    print(v)
print("-" * 20, "config", "-" * 20)
print(snapshot.config)
print("-" * 20, "next", "-" * 20)
print(snapshot.next)